In [1]:
KAFILE = 'cs-export-2020-07-14T11-18-37+0000.tsv'
KSFILE = 'ks_videos.tsv'

In [2]:
ks_ytids = set()
# Also prints duplicate videos
with open(KSFILE, 'r') as f:
    for line in f:
        id, title, ytid, ytid_original, ka_url = line.split('\t')
        if ytid_original != '' and ytid_original != '""':
            ks_ytids.add(ytid_original)
        if ytid in ks_ytids:
            print(line)
        else:
            ks_ytids.add(ytid)

10	Odčítání z hlavy	GBtcGO44e-A	GBtcGO44e-A	

2805	Prvočísla	HYBSlf0P1DA	mIStB5X4U8M	https://cs.khanacademy.org/v/prime-numbers

2818	Vyjádření proměnné z výrazu	U-ESXnneNhc	AJNDeVt9UOo	https://cs.khanacademy.org/v/variables-and-expressions-1

2841	Prvočíselný rozklad	Ala4BZjBbkc	ZKKDTfHcsG0	https://cs.khanacademy.org/v/prime-factorization

2994	Úvod do goniometrie	YD3E_L7YNRM	Jsiy4TxgIME	https://cs.khanacademy.org/v/basic-trigonometry

3443	Nerovnice s absolutní hodnotou	TvUCe6Bomy4		

3659	5 úžasných číselných soustav	RoMGatx4IRQ	RoMGatx4IRQ	

3750	Porovnávání celých čísel	J46hYYrkXyM	9Jg5S7F2SMQ	

3761	Dělení dvouciferným číslem - řešený příklad	WmbyKBtm9us	uCBm8iDyg1s	

3763	Slovní úlohy na odčítání do 10	G-rfJB0Zl3A	qSkpZswoZTc	

3767	Postupy pro sčítání čísel do 100	e5MDP7hgruY	kMHFY6ZrKbU	

3774	Slovní úloha: Sčítání a odčítání 2ciferných čísel	ZDripoF5__U	qyvwN-2PoXAd	



In [3]:
cols = {
    'kind': 0,
    'id': 2,
    'url': 9,
    'ytid': 16,
    'translated_ytid': 27,
    'listed': 11,
}

In [128]:
SQL_TEMPLATE = "UPDATE contents SET ka_url = '%s' WHERE youtube_id IN ('%s', '%s') OR youtube_id_original IN ('%s', '%s');\n"

sql_commands = []
ka_ids = set()
with open(KAFILE, 'r') as f:
    for line in f:
        l = line.split('\t')
        # Skip everything but videos
        if l[cols['kind']].lower() != 'video':
            continue
        # Skip unlisted videos
        # MUST BE HERE, BEFORE IDs
        if l[cols['listed']].lower() == 'true':
            listed = True
        else: 
            listed = False
        if not listed:
            continue
        
        # Skip duplicate videos in KA content export
        id = l[cols['id']]
        if id in ka_ids:
            continue
        else:
            ka_ids.add(id)
  
        ytid = l[cols['ytid']]
        translated_ytid = l[cols['translated_ytid']]
        if translated_ytid == '' or translated_ytid is None:
            translated_ytid = ytid
        canonical_url = l[cols['url']]
        if ytid in ks_ytids or ytid_translated in ks_ytids:
            sql_cmd = SQL_TEMPLATE % (canonical_url, ytid, translated_ytid, ytid, translated_ytid)
            sql_commands.append(sql_cmd)

In [129]:
print(sql_commands[0])
print(len(sql_commands))

UPDATE contents SET ka_url = 'https://cs.khanacademy.org/science/cosmology-and-astronomy/universe-scale-topic/scale-earth-galaxy-tutorial/v/scale-of-the-large' WHERE youtube_id IN ('5FEjrStgcF8', '5FEjrStgcF8') OR youtube_id_original IN ('5FEjrStgcF8', '5FEjrStgcF8');

845


In [130]:
SQL_FNAME = 'update_ka_urls.sql'
with open(SQL_FNAME, 'w') as f:
    for cmd in sql_commands:
        f.write(cmd)

In [79]:
if '' in ks_ytids:
    print('empty_string')

In [113]:
# Can also be computed with this AWK one-liner:
# awk -F"\t" '{if($1=="Video" && $12=="True") print $3}' cs-export-2020-07-14T11-18-37+0000.tsv | sort -n | uniq | wc -l

# for calculating all content types at once
# awk -F"\t" '{if($12=="True") print $1,$3}' cs-export-2020-07-14T11-18-37+0000.tsv | sort -n | uniq | awk '{print $1}' | uniq -c
def count_unique_content(fname, ct_kind):
    count = 0
    ka_ids = set()
    with open(fname, 'r') as f:
        for line in f:
            l = line.split('\t')
            id = l[cols['id']]
            kind = l[cols['kind']]
            if l[cols['listed']].lower() == 'true':
                listed = True
            else: 
                listed = False
                
            if kind.lower() != ct_kind.lower():
                continue
                
            if not listed:
                continue
                
            if id in ka_ids:
                continue
            
            ka_ids.add(id)
            count += 1

    return count

def count_all(fname):
    kinds = ('video', 'exercise', 'talkthrough', 'Challenge', 'Interactive', 'Project',
             'lesson', 'unit', 'course', 'domain')
    
    counts = {}
    for kind in kinds:
        counts[kind] = count_unique_content(fname, kind)

    return counts

In [114]:
count_all(KAFILE)

{'video': 1630,
 'exercise': 678,
 'talkthrough': 32,
 'Challenge': 29,
 'Interactive': 26,
 'Project': 11,
 'lesson': 697,
 'unit': 137,
 'course': 29,
 'domain': 5}

In [121]:
# Shit, need to FIX videos that I redirected but that are in fact unlisted
SQL_TEMPLATE = "UPDATE contents SET ka_url = NULL WHERE youtube_id IN ('%s', '%s') OR youtube_id_original IN ('%s', '%s');\n"
sql_commands = []
ka_ids = set()
with open(KAFILE, 'r') as f:
    for line in f:
        l = line.split('\t')
        # Skip everything but videos
        if l[cols['kind']].lower() != 'video':
            continue
        # Skip duplicate videos in KA content export
        id = l[cols['id']]
        if id in ka_ids:
            continue
        else:
            ka_ids.add(id)
  
        # Skip LISTED videos
        if l[cols['listed']].lower() == 'true':
            listed = True
        else: 
            listed = False
        if listed:
            continue
            
        ytid = l[cols['ytid']]
        translated_ytid = l[cols['translated_ytid']]
        if translated_ytid == '' or translated_ytid is None:
            translated_ytid = ytid
        canonical_url = l[cols['url']]
        if ytid in ks_ytids or ytid_translated in ks_ytids:
            sql_cmd = SQL_TEMPLATE % (ytid, translated_ytid, ytid, translated_ytid)
            sql_commands.append(sql_cmd)

In [124]:
len(sql_commands)+468

1915

In [125]:
SQL_FNAME = 'fix_unlisted_ka_urls.sql'
with open(SQL_FNAME, 'w') as f:
    for cmd in sql_commands:
        f.write(cmd)

In [132]:
!ls

cs-export-2020-07-14T11-18-37+0000.tsv	ks_videos.tsv
explore.ipynb				ks_visible_content.csv
fix_unlisted_ka_urls.sql		o
hide_videos.sql				update_ka_urls_19-07-2020.sql


## Hide videos not in KA anymore

But be careful, some videos are not from KA! E.g. Crash Course and others

In [5]:
ka_ytids = set()
EN_KAFILE = '/home/hollas/khan/csv-export/tmp/tsv_download.en.tsv'
with open(EN_KAFILE, 'r') as f:
    for line in f:
        l = line.split('\t')
        # Skip everything but videos
        if l[cols['kind']].lower() != 'video':
            continue
            
        ka_ytids.add(l[cols['ytid']])
        #ka_ytids.add(l[cols['translated_ytid']])
        # Here we do not skip unlisted

In [22]:
KSFILE_NEW = 'ks_visible_content_new.csv'
KSFILE_NEW = 'ks_visible_videos_outside_blocks.csv'
ks_ids_to_hide = set()
with open(KSFILE_NEW, 'r', encoding='utf-8-sig') as f:
    for line in f:
        id, title, ytid, ytid_original, ka_url, = line.split('\t')
        if ytid_original not in ka_ytids and ytid not in ka_ytids:
            ks_ids_to_hide.add(int(id))
len(ks_ids_to_hide)

162

In [23]:
SQL_TMPL = "UPDATE contents SET hidden = 't' WHERE id = %d;\n"
with open('hide_removed_orphans.sql', 'w') as f:
    for id in ks_ids_to_hide:
        f.write(SQL_TMPL % id)
    